In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import *

1.8.1
0.8.0a0+e4e171a


In [7]:
cfg = Config(
    noise_nsr_dbs = [30, 20, 10, 3],
    site='SSW', 
    use_neptune=True, 
    n_epochs=100,
    bs=32, 
    lr=1e-3, 
    model='densenet-mixedspec',
    # noise_dir = 'data/birdclef-2021/train_soundscapes.wav',
)

In [8]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 10752,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [30, 20, 10, 3],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': True,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.001,
 'n_epochs': 100,
 'model': 'densenet-mixedspec',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.CosineAnnealingLR',
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat

In [9]:
prep = torch.nn.Sequential(
    MixedSig2Spec(cfg),
)

main_model = torch.hub.load('pytorch/vision:v0.9.0', 'densenet121', pretrained=True)
main_model.classifier = torch.nn.Linear(in_features=1024, out_features=len(cfg.labels), bias=True)

model = torch.nn.Sequential(prep, main_model)
for param in model.parameters():
    param.requires_grad = False

for m in [model[1].features.denseblock3, model[1].features.transition3, model[1].features.denseblock4, model[1].features.norm5, model[1].classifier]:
    for param in m.parameters():
        param.requires_grad = True
    



model = model.cuda()

Using cache found in /home/koen/.cache/torch/hub/pytorch_vision_v0.9.0


In [10]:
lrn = Learner("densenet-mixed-spec-half-frozen", cfg, model)

In [17]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-48


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
lrn.name

'densenet-mixed-spec-half-frozen-20210502-054933'

In [15]:
lrn.name = 'densenet-mixed-spec-continue-unfrozen-20210502-072700'

In [16]:
for param in model.parameters():
    param.requires_grad = True

lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-47


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-02 09:48:05 epoch:  99 train loss: 0.080075 train f1: 0.724623 valid loss: 0.121831 valid f1: 0.252381


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-02 09:49:30 epoch: 100 train loss: 0.077947 train f1: 0.738476 valid loss: 0.095296 valid f1: 0.364469


In [18]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,398,30436,2394,372,0.223470
2,0.2,306,31900,930,464,0.305085
3,0.3,257,32370,460,513,0.345662
4,0.4,229,32601,229,541,0.372964
5,0.5,199,32707,123,571,0.364469
6,0.6,168,32760,70,602,0.333333
7,0.7,143,32792,38,627,0.300736
8,0.8,123,32812,18,647,0.270033
9,0.9,97,32819,11,673,0.220957


In [19]:
lrn.save_checkpoint(100, 0.0952, 0.3644)

In [20]:
lrn.load_checkpoint('densenet-mixed-spec-continue-unfrozen-best-20210502-072700')

{'epoch': 86,
 'valid_loss': 0.08637355826795101,
 'valid_score': 0.42659279704093933}

In [21]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,520,29258,3572,250,0.213904
2,0.2,412,31873,957,358,0.385227
3,0.3,322,32530,300,448,0.462644
4,0.4,280,32695,135,490,0.472574
5,0.5,231,32748,82,539,0.426593
6,0.6,199,32781,49,571,0.390963
7,0.7,168,32807,23,602,0.349636
8,0.8,136,32815,15,634,0.295331
9,0.9,104,32824,6,666,0.236364
